# Ewald sum

References:
- "Computer Simulation of Liquids" by Michael P. Allen and Dominic J. Tildesley (2nd edition, Oxford University Press, 2017)
- https://github.com/Allen-Tildesley

In [ ]:
import numpy as np
import torch
from scipy.special import erfc

def distance( x ):
    return torch.norm(x[None, :, :] - x[:, None, :], dim=-1)

def qpairs( q ):
    return q[None, :]*q[:, None]

# Real Space

In [ ]:
def rwald( x, q, sigma=0.1178511 ):
    """Returns the r-space part of energy of ewald summation
    
    Arguments:
        x (float): position vectors (dim = n x 3)
        q (int): charges (dim = n)
        sigma (float): sqrt(variance), width of gaussian
    
        n (int): number of particles
    
    Output:
        e_short (float): short range part of ewald potential
    """
    epsilon = 8.854187817e-12
    #x = torch.tensor(x, dtype=np.float32)
    #q = torch.tensor(q, dtype=np.float32)
    n, d = list(x.size())
    assert n == q.shape[0], 'dimension error: q needs n entries'
    
    e_short = 0.
    
    r = distance(torch.Tensor(x))
    qiqj = qpairs(torch.Tensor(q))
    r_invers = r.clone()
    r_invers[r_invers!=0] = 1/r_invers[r_invers!=0]
    return torch.sum(
        qiqj * r_invers * erfc( r / (np.sqrt(2) * sigma) )
        ) / (2 * np.pi * epsilon)

In [ ]:
charge = torch.tensor([-1., 2, -1, 3])
position = torch.tensor([[0., 0.], [0., 1.], [1., 1.], [1., 0.]])
#qiqj = qpairs(charge)
#print(qiqj)
#dist = distance(position)
#print(dist)
#dist[dist!=0] = 1/dist[dist!=0]
#print(qiqj*dist)
#print(torch.sum(qiqj))
k = torch.tensor([[2., 3], [-1., -3]])
r_ab = position[None, :, :] - position[:, None, :]
#print(k)
#print(r_ab)
#print(k[:, None, None]*r_ab)
#print(torch.sum(k[:, None, None]*r_ab, dim=-1)) 

In [ ]:
print(rwald(position, charge))
#a = np.array([[.1, .2, .4], [.8, 1.6, 3.2]])
#print(a)
#print(erfc(a*6))

# Fourier space

In [ ]:
import numpy as np
import torch
from scipy.special import erfc

def dx( x ):
    return x[None, :, :] - x[:, None, :]

def distance( x ):
    return torch.norm(x[None, :, :] - x[:, None, :], dim=-1)

def qpairs( q ):
    return q[None, :]*q[:, None]

In [ ]:
def nk( maxki ):
    """Combinatorics, 
    returns number of combis for lattice vectors in 3D
    """
    mitnull = (maxki + 1) ** 3 * 2 ** 3
    korr = 7 + sum([30 + i * 24 for i in range(maxki)])
    
    return mitnull - korr

def kvec( maxki=3, maxnk=300, maxr=0):
    """Returns k-vectors for k-part of ewald
    
    Arguments:
        maxki (int): max value for kx, ky, kz
        maxnk (int): max # of k vectors, unrestrict with maxnk=0 
        maxr  (int): max length of k, unrestrict with maxr=0
    
    Output:
        k (float): k vectors (dim = 3 x maxnk)
    """
    vector = np.zeros([nk(maxki), 3], dtype=np.float32)
    cnt = 0
    for i in range(maxki + 1):
        for j in range(maxki + 1):
            for k in range(maxki + 1):
                if (maxr == 0 or maxr >= i**2+j**2+k**2):
                    vector[cnt] = [i, j, k]
                    cnt += 1
                    if (i > 0):
                        vector[cnt] = [-i, j, k]
                        cnt += 1
                    if (j > 0):
                        vector[cnt] = [i, -j, k]
                        cnt += 1
                    if (k > 0):
                        vector[cnt] = [i, j, -k]
                        cnt += 1
                    if (i > 0 and j > 0):
                        vector[cnt] = [-i, -j, k]
                        cnt += 1
                    if (i > 0 and k > 0):
                        vector[cnt] = [-i, j, -k]
                        cnt += 1
                    if (j > 0 and k > 0):
                        vector[cnt] = [i, -j ,-k]
                        cnt += 1
                    if (i > 0 and j > 0 and k > 0):
                        vector[cnt] = [-i, -j, -k]
                        cnt += 1
    if (maxnk > 0 ):   
        indexlist = np.argsort(np.linalg.norm(vector,axis=-1))
        vector = torch.from_numpy(vector[indexlist])
        vector = vector[0:maxnk]
    else:
        vector = torch.from_numpy(vector[0:cnt])
    return vector

In [ ]:
k = kvec() # default k: 300 vectors
print(len(k), k[-1])
k = kvec(10, 0) # unlimited number of k vectors
print(len(k), k[-1])
k = kvec(5, 1) # specific number of k vectors
print(len(k), k[-1])
k = kvec(5, 0, 15) # max len(k)
print(len(k), k[-1])

In [ ]:
def kvec_time(ki):
    import time
    t1 = time.perf_counter()
    k = kvec(ki, 0)
    t2 = time.perf_counter()
    return ki, len(k), float(torch.norm(k[-1])), t2-t1

for i in range(9):
    print(kvec_time(i**2))

# Structure Factor squared

$$\mathrm{sfac} = |S(\textbf{k})|^2 = 
\sum_{a,b}^N q_a q_b \cos(<\textbf{k}, r_{ab}>)$$

- saved as vector ( dim(sfactor) = dim(k) )

In [ ]:
def sfac( x, q, k ):
    """Structure factor squared
    """
    return torch.sum(
        torch.sum( qpairs( q ) * np.cos( 
        torch.sum( k[:, None, None] * dx( x ), 
                  dim=-1 )), dim=-1), dim=-1)

# K-space
$$ E_L = \frac{1}{2V\varepsilon_0} 
    \sum_\textbf{k} |S(\textbf{k})|^2 \frac{\exp
    \left(-\sigma^2 k^2\right)}{k^2}$$
    
$$ \mathrm{with} \quad |\textbf{k}| = k $$

In [ ]:
def kwald( x, q, lbox=1, kmax=3 ):
    """Returns the fourier space part of energy of ewald summation
    
    Arguments:
        x    (float): position vectors (dim = n x 3)
        q      (int): charges (dim = n)
        lbox (float): box length
        kmax   (int): max value for k_i \in {k_x, k_y, k_z}
        
        n := # of particles
        sigma (float): sqrt(variance), width of gaussian
    
    Output:
        e_l   (float): long range part of ewald potential
    """
    sigma = lbox/10.
    eps = 1 # 8.854187817e-12
    
    k = kvec(kmax, 0)*(2*np.pi/lbox)
    s = sfac(x, q, k)
    
    ksq = torch.sum(k**2, dim=-1)
    kinv = ksq.clone()
    kinv[kinv != 0] = 1 / kinv[kinv != 0]
    frac = np.exp(-sigma ** 2 * ksq / 2) * kinv
    
    return torch.sum(s * frac) / (2 * lbox ** 3 * eps)

In [ ]:
pos = torch.tensor([[0., 0, 3],
        [0, 1, 3],
        [1, 1, -3],
        [1, 0, -3]])
cha = torch.tensor([-1., 2, -1, 3])

In [ ]:
import time

n = 50 # number of particles
position = torch.from_numpy(
    np.random.rand(n, 3).astype(dtype=np.float32))
charge = torch.from_numpy(
    np.random.choice([-1.,1], n).astype(dtype=np.float32))

for i in range(1, 6):
    t1 = time.perf_counter()
    e_l = float(kwald(position, charge, 1, i**2))
    t2 = time.perf_counter()
    print(i**2, e_l, t2-t1)